In [1]:
import scipy.linalg as la
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
import numpy as np
from helper_functions.grape_functions import *
from main_grape.Grape import Grape
%pylab inline



Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
qubit_num = 4


#Defining time scales
#total_time = qubit_num * 5.0
#steps = qubit_num * 500

total_time = 10.0
steps = 1000

state_transfer = False
#Defining H0
qubit_state_num = 2
freq_ge = 3.9#GHz
g = 2*np.pi*0.5 #GHz

ens = np.array([ 2*np.pi*ii*(freq_ge) for ii in np.arange(qubit_state_num)])
Q_x   = np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)+np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1)
Q_y   = (0+1j) *(np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)-np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1))
Q_z   = np.diag(np.arange(0,qubit_state_num))
I_q = np.identity(qubit_state_num)
H_q = np.diag(ens)

X = multi_kron(Q_x,qubit_num)
Z = multi_kron(Q_z,qubit_num)
Y = multi_kron(Q_y,qubit_num)

g_op = nn_chain_kron(Q_x, I_q, qubit_num, qubit_state_num)

# g_op_test = np.kron(np.kron(np.kron(Q_x,Q_x),I_q),I_q) + np.kron(np.kron(np.kron(I_q,Q_x),Q_x),I_q) + np.kron(np.kron(np.kron(I_q,I_q),Q_x),Q_x)
# print np.allclose(g_op, g_op_test)

H0 = kron_all(H_q,qubit_num, I_q) + g * g_op

# Defining dressed info
is_dressed = True
w_c, v_c, dressed_id = get_dressed_info(H0)
dressed_info = {'dressed_id':dressed_id, 'eigenvectors':v_c, 'eigenvalues':w_c,'is_dressed':is_dressed}


#Defining Concerned states (starting states)

psi0=[0]

all_ones_index = 0
for ii in range (qubit_num):
    all_ones_index = all_ones_index + qubit_state_num**ii
#Defining states to include in the drawing of occupation
states_draw_list = [0,all_ones_index]
states_draw_names = ['00..0','11..1']

#Defining U (Target)

if is_dressed:
    
    gg = get_state_index(0,dressed_id) #[ee]
    ee = get_state_index(all_ones_index,dressed_id) #[eg]
    
    target_vec = np.zeros(qubit_state_num**qubit_num,dtype=complex)
    target_vec = target_vec +  (1)/np.sqrt(2)*v_c[:,gg] + (1)/np.sqrt(2)*v_c[:,ee]
    
    U = [target_vec]

else:   
    target_vec = np.zeros(qubit_state_num**qubit_num,dtype=complex)
    target_vec[0]=(1)/np.sqrt(2)
    target_vec[all_ones_index]=1/np.sqrt(2)

    U = [target_vec]



#Defining U0 (Initial)
q_identity = np.identity(qubit_state_num**qubit_num)
U0= q_identity

#Defining control Hs

Hops = []
Hnames = []
ops_max_amp = []
Hops,Hnames,ops_max_amp = append_separate_krons(Q_x,'x',qubit_num,qubit_state_num,Hops,Hnames,ops_max_amp,amp=1.0)
#Hops,Hnames,ops_max_amp = append_separate_krons(Q_z,'z',qubit_num,qubit_state_num,Hops,Hnames,ops_max_amp,amp=1.0)

print Hnames

#Defining convergence parameters
max_iterations = 1000
decay = max_iterations/2
convergence = {'rate':0.11, 'update_step':50, 'max_iterations':max_iterations,\
               'conv_target':1e-3,'learning_rate_decay':decay}

reg_coeffs = {'envelope':0.01,'dwdt':0.001,'d2wdt2':0.0001*0.001}


True
['xiii', 'ixii', 'iixi', 'iiix']


In [ ]:
 #Extra options
#reg_coeffs = reg_coeffs,draw = [states_draw_list,states_draw_names] ,forbidden = states_forbidden_list,multi_mode = multi_mode,
#maxA = ops_max_amp,use_gpu = True

uks,U_final = Grape(H0,Hops,Hnames,U,total_time,steps,psi0,convergence=convergence, draw = [states_draw_list,states_draw_names],  state_transfer=True,
                    show_plots = False, 
       unitary_error = 1e-6, method = 'L-BFGS-B',switch = False,  maxA=ops_max_amp,\
                   dressed_info = dressed_info, reg_coeffs=reg_coeffs, file_name='spin_chain_ghz_N%d' %(qubit_num),
                    data_path = '/home/nelson/Simulations/GRAPE-Data/spin_chain_ghz')
      
      

In [ ]:
import numpy as np
import h5py

with h5py.File('/home/nelson/Simulations/GRAPE-Data/spin_chain_ghz/00009_spin_chain_ghz_N2.h5','r') as hf:
    print('List of arrays in this file: \n', hf.keys())
    steps = np.array(hf.get('steps'))
    print steps
    total_time = np.array(hf.get('total_time'))
    print total_time
    print np.array(hf.get('states_concerned_list'))
    print np.array(hf.get('iteration'))
    print np.array(hf.get('error'))
    print len(np.array(hf.get('iteration')))
    print len(np.array(hf.get('error')))
    print np.array(hf.get('inter_vecs_mag_squared')).shape
    print np.array(hf.get('uks')).shape
    print np.array(hf.get('run_time'))
    
    inter_vecs_real = np.array(hf.get('inter_vecs_real'))
    inter_vecs_imag = np.array(hf.get('inter_vecs_imag'))
    
    inter_vecs = inter_vecs_real + 1j * inter_vecs_imag
    
    print inter_vecs.shape
    
    final_vec = inter_vecs[0][0][:,-1]
    
    print final_vec.shape
    
    print final_vec
    
    print U[0]
    
    print np.abs(np.inner(np.conjugate(U[0]),final_vec))
    
    print np.square(np.abs(np.inner(np.conjugate(U[0]),final_vec)))
    
    print np.abs(final_vec)
    
    psi_1_real = np.real(final_vec)
    psi_1_imag = np.imag(final_vec)
    psi_2_real = np.real(U[0])
    psi_2_imag = np.imag(U[0])
    
    
    ac = psi_1_real*psi_2_real
    bd = psi_1_imag*psi_2_imag
    bc = psi_1_imag*psi_2_real
    ad = psi_1_real*psi_2_imag
    
    error = np.square(np.sum(ac)+ np.sum(bd)) + np.square(np.sum(bc) - np.sum(ad))
    
    
    v_sorted=sort_ev(v_c,dressed_id)
    
    dressed_U0= np.dot(np.transpose(v_sorted),U[0])
    
    print 1-np.square(np.abs(np.inner(np.conjugate(dressed_U0),final_vec)))
    

In [ ]:
0.0153037262851/2.19345093e-05